This notebook is a first look at the structure of NRES spectrum files.

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

Here are all the files.

In [ ]:
filenames = glob.glob('NRES/*e92-1d.fits.fz')
filenames

Let's load one file and look at its structure.

In [ ]:
f = filenames[6]
hdu = fits.open(f)
hdu.info()
hdu['PRIMARY'].header['MJD-OBS']

The `PRIMARY` extension doesn't have any `.data` associated with it, but it has a header that's full of super useful information about the observation.

In [ ]:
hdu['PRIMARY'].header

The data are all contained in the `SPECTRUM` extension. There are a [few keywords in the data for each spectrum](https://banzai-nres.readthedocs.io/en/latest/banzai_nres/data-products.html#extracted-spectra) that we will be most interested in. 

+ ‘id’: Integer ID of the trace in this row. This corresponds to the number in the TRACE extension of the 2-D calibration products.
+ ‘order’: Physical dispersion order of this trace.
+ ‘fiber’: Fiber ID. The calibration fiber is always 1. The science fiber is either 0 or 2. The science fiber on the target can be determined from the ‘SCIFIBER’ keyword in the header.
+ ‘wavelength’: Wavelength per pixel for this trace (Angstroms).
+ ‘flux’: Extracted flux (counts). No blaze correction or continuum normalization is performed on this data.
+ ‘uncertainty’: Formal uncertainty of the extracted flux propagated from the previous processing stages. No blaze correction or continuum normalization is performed on this data.
+ ‘blaze’: Extracted estimate of the blaze function from the quartz lamp flat field.
+ ‘blaze_error’: Formal uncertainties on the blaze due to the Poisson statistics in the quartz lamp flat field.
+ ‘mask’: Bad pixels have values greater than zero.
+ ‘normflux’: Extracted flux after the blaze and continuum have been divided out.
+ ‘normuncertainty’: Formal uncertainty of the extracted pixels after scaling by the blaze and continuum.

In [ ]:
len(hdu['SPECTRUM'].data)

The spectrum file contains both the actual science spectrum (the flux of the star) and the calibration lamp spectrum (an internal calibration lamp used to figure out the wavelength solution of the spectrograph). Let's trim down to just the science spectra.

In [ ]:
# which fiber contains the science spectrum? 
# (the other one contains the calibration data)
science_fiber_id = hdu['PRIMARY'].header['SCIFIBER']
is_science = hdu['SPECTRUM'].data['FIBER'] == science_fiber_id
data = hdu['SPECTRUM'].data[is_science]

In [ ]:
len(data)

In [ ]:
data.columns

Each row of `data` now corresponds to an individual echelle order. If we pick any one particular row, we'll get just data associated with that order. 

In [ ]:
# pick one particular row (indexed from 0)
i = 0

# trim to just that one row
this_order = data[i]

In [ ]:
this_order['ORDER']

Additionally, there is a `mask` defining which pixels within the row are "good" or not. Let's trim to just the data that aren't definitely bad:

In [ ]:
# what does the "good data" mask look like? 
plt.figure(figsize=(8,3), dpi=300)
plt.plot(this_order['MASK']);